# Predict use GMMVSG

In [ ]:
# Import library
import pandas as pd                                              
import numpy as np                                       
import matplotlib.pyplot as plt   
import seaborn as sns
import scipy.stats as st

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from scipy.stats import spearmanr
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error  
from numpy import mean
from numpy import sqrt

In [ ]:
# Import data
data = pd.read_excel('Heterocyclic.xlsx')
x = data.drop(columns='IE')
y = data['IE']

# Standardize the data
scaler = RobustScaler()
x = scaler.fit_transform(x)

In [ ]:
# define the range of number of components to try
n_components_range = range(1, 12)

# calculate AIC for each number of components
aics = []
for n_components in n_components_range:
    gmm = GaussianMixture(n_components=n_components, covariance_type='full')
    gmm.fit(x)
    aics.append(gmm.aic(x))

# select the optimal number of components based on the AIC
best_n_components = n_components_range[np.argmin(aics)]

# fit GMM on the dataset with the optimal number of components
gmm = GaussianMixture(n_components=best_n_components, covariance_type='full')
gmm.fit(x)

# generate virtual samples for the input features using GMM
n_samples = 1000
x_vs, _ = gmm.sample(n_samples)

# Concatenate original and virtual samples
x_aug = np.concatenate([x, x_vs])

# inverse transform the scaled data back to the original scale
x_new = scaler.inverse_transform(x_aug)

# generate virtual samples for the target variable using linear regression
model = GradientBoostingRegressor()
model.fit(x, y)
y_vs = model.predict(x_new)
y_new = y_vs

data_new = pd.DataFrame(np.column_stack([x_new, y_new]), columns=list(data.columns))

In [ ]:
# Histogram
data_new.hist(bins=20, figsize=(20,15))
plt.show()

In [ ]:
# Extract the feature and target variables
homo = data_new['HOMO']
ie = data_new['IE']

plt.figure()
plt.scatter(con, ie, c='b')
plt.title('Distribution of HOMO')
plt.xlabel('HOMO')
plt.ylabel('IE')

In [ ]:
kolom_x = ['HOMO', 'LUMO', 'Gap Energy', 'Dipole Moment', 'Ionization Potential',
       'Electron Affinity', 'Electronegativity', 'Global Hardness',
       'Global Softness', 'Electrophilicity',
       'Fraction of electron transferred', 'IE']

for i in range(len(kolom_x)):
    # calculate spearman's correlation
    x = data_new[kolom_x[i]]
    y = y_new
    coef, p = spearmanr(x, y)
    print(kolom_x[i],': ', np.round(coef,3))
    # interpret the significance
    alpha = 0.05
    
    if p > alpha:
        print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
        print()
    else:        
        print('Samples are correlated (reject H0) p=%.3f' % p)
        print()

In [ ]:
# CV 2
kfold = KFold(n_splits=5, shuffle=True, random_state=1)
for train, test in kfold.split(x_new, y_new):
    x_train, x_test = x_new[train], x_new[test]
    y_train, y_test = y_new[train], y_new[test]

# Model
model = GradientBoostingRegressor()
model.fit(x_train, y_train)

# Prediksi
y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)
        
print('Training')
print('R^2 :', r2_score(y_train, y_pred_train))
print('MAE :', mean_absolute_error(y_train, y_pred_train))
print('MAPE:', mean_absolute_percentage_error(y_train, y_pred_train))
print('MSE :', mean_squared_error(y_train, y_pred_train))
print('RMSE:', np.sqrt(mean_squared_error(y_train, y_pred_train)))
print('================================')
print('Testing')
print('R^2 :', r2_score(y_test, y_pred_test))
print('MAE :', mean_absolute_error(y_test, y_pred_test))
print('MAPE:', mean_absolute_percentage_error(y_test, y_pred_test))
print('MSE :', mean_squared_error(y_test, y_pred_test))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred_test)))

# Plot
xline   = np.array(np.linspace(np.min(y)-0.1, np.max(y)+0.1, 150))
yline   = np.array(np.linspace(np.min(y)-0.1, np.max(y)+0.1, 150))

plt.figure(figsize=(8,4))
plt.scatter(y_train, y_pred_train, c='blue', label = "Training")
plt.scatter(y_test, y_pred_test, c='red', label = "Testing")
plt.plot(xline, yline, color="black", label = "actual line", linestyle='dotted')   
plt.legend(loc='best')
plt.show()